## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Grovenor House\r2627 S Bayshore Dr 2403\rCocon...,5/1/2023,16,"$4,250,000.00",2640,"$1,609.85",Anthony Lauria,"Compass Florida, LLC",Geane Brito,"Compass Florida, LLC",Grovenor House 2627 S Bayshore Dr 2403 Coconut...,Grovenor House,2627 S Bayshore Dr 2403 Coconut Grove,"(25.7322298, -80.2348919)",25.732230,-80.234892
2,1100 West Condo Mondrian\r1100 West Ave TS-2/3...,5/4/2023,250,"$3,900,000.00",4120,$946.60,Juliana Ospina,"Compass Florida, LLC",Salesperson Non Member,Non-MLS Member,1100 West Condo Mondrian 1100 West Ave TS-2/3 ...,,1100 West Condo Mondrian 1100 West Ave TS-2/3 ...,"(25.7817857, -80.1428147)",25.781786,-80.142815
3,Palau Sunset Harbour\r1201 20th St 205\rMiami ...,5/2/2023,1,"$3,650,000.00",1820,"$2,005.49",John Sandberg,"Compass Florida, LLC",Salesperson Non Member,Non-MLS Member,Palau Sunset Harbour 1201 20th St 205 Miami Beach,Palau Sunset Harbour,1201 20th St 205 Miami Beach,"(25.795711, -80.14189499999999)",25.795711,-80.141895
4,Chateau Beach\r17475 Collins Ave 1901\rSunny I...,5/1/2023,39,"$3,300,000.00",2703,"$1,220.87",Lana Bell,Douglas Elliman,Lana Bell,Douglas Elliman,Chateau Beach 17475 Collins Ave 1901 Sunny Isl...,Chateau Beach,17475 Collins Ave 1901 Sunny Isles Beach,"(25.9378917, -80.12093329999999)",25.937892,-80.120933
5,Murano Grande\r400 Alton Rd 1201\rMiami Beach,5/3/2023,264,"$3,200,000.00",2003,"$1,597.60",Billy Hernandez,Douglas Elliman,Nancy Batchelor PA,"Compass Florida, LLC",Murano Grande 400 Alton Rd 1201 Miami Beach,Murano Grande,400 Alton Rd 1201 Miami Beach,"(25.7731379, -80.14060429999999)",25.773138,-80.140604
6,Lake Villa One Condo\r721 Crandon Blvd PH8\rKe...,5/1/2023,122,"$2,245,000.00",2683,$836.75,Coralia Braverman,Florida Capital Realty,Michelle Shurtleff,"Compass Florida, LLC",Lake Villa One Condo 721 Crandon Blvd PH8 Key ...,Lake Villa One Condo,721 Crandon Blvd PH8 Key Biscayne,"(25.6879859, -80.16285909999999)",25.687986,-80.162859
7,Carbonell Condo\r901 Brickell Key Blvd 3708\rM...,5/3/2023,24,"$2,102,000.00",2860,$734.97,April Feeser,Realhome Service & Solutions I,Estela Serrano,"Serrano & Associates Real Estate, LLC",Carbonell Condo 901 Brickell Key Blvd 3708 Miami,Carbonell Condo,901 Brickell Key Blvd 3708 Miami,"(25.7691361, -80.1869402)",25.769136,-80.186940
8,TDR Tower II Condo\r15901 Collins Ave 4104\rSu...,5/3/2023,182,"$2,100,000.00",2327,$902.45,Olena Sartan,Miami Residence Inc.,Marco Cunha,Prestige Luxury Real Estate,TDR Tower II Condo 15901 Collins Ave 4104 Sunn...,TDR Tower II Condo,15901 Collins Ave 4104 Sunny Isles Beach,"(25.9240235, -80.1218071)",25.924024,-80.121807
9,The Pinnacle Condo\r17555 Collins Ave PH-7\rSu...,5/1/2023,32,"$2,000,000.00",1840,"$1,086.96",Tomer Kort,"Brokerage S. Florida Real Estate, LLC.",Hector Stracuzzi,Beachfront Realty Inc,The Pinnacle Condo 17555 Collins Ave PH-7 Sunn...,The Pinnacle Condo,17555 Collins Ave PH-7 Sunny Isles Beach,"(25.9392319, -80.120541)",25.939232,-80.120541
10,02977 Bridgeport Condo\r2977 Bridgeport Ave 29...,5/1/2023,56,"$1,965,000.00",2802,$701.28,Monique Pennen-Wills PA,BHHS EWM Realty,Axelle Gault,"Global Axellence, Inc",02977 Bridgeport Condo 2977 Bridgeport Ave 297...,,02977 Bridgeport Condo 2977 Bridgeport Ave 297...,"(25.7360738, -80.2431314)",25.736074,-80.243131


In [9]:
df.at[2,'building_name']=('Mondrian')
df.at[2,'address_only']=('West Condo Mondrian')

df.at[10,'building_name']=('Bridgeport')
df.at[10,'address_only']=('2977 Brigeport Ave')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 30th - May 6th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [21]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon,int_Sale_Price,RANK,COLOR
1,Grovenor House\r2627 S Bayshore Dr 2403\rCocon...,5/1/2023,16,"$4,250,000.00",2640,1609.85,Anthony Lauria,"Compass Florida, LLC",Geane Brito,"Compass Florida, LLC",Grovenor House 2627 S Bayshore Dr 2403 Coconut...,Grovenor House,2627 S Bayshore Dr 2403 Coconut Grove,"(25.7322298, -80.2348919)",25.732230,-80.234892,4250000.0,1,orange
2,1100 West Condo Mondrian\r1100 West Ave TS-2/3...,5/4/2023,250,"$3,900,000.00",4120,946.60,Juliana Ospina,"Compass Florida, LLC",Salesperson Non Member,Non-MLS Member,1100 West Condo Mondrian 1100 West Ave TS-2/3 ...,Mondrian,West Condo Mondrian,"(25.7817857, -80.1428147)",25.781786,-80.142815,3900000.0,2,blue
3,Palau Sunset Harbour\r1201 20th St 205\rMiami ...,5/2/2023,1,"$3,650,000.00",1820,2005.49,John Sandberg,"Compass Florida, LLC",Salesperson Non Member,Non-MLS Member,Palau Sunset Harbour 1201 20th St 205 Miami Beach,Palau Sunset Harbour,1201 20th St 205 Miami Beach,"(25.795711, -80.14189499999999)",25.795711,-80.141895,3650000.0,3,blue
4,Chateau Beach\r17475 Collins Ave 1901\rSunny I...,5/1/2023,39,"$3,300,000.00",2703,1220.87,Lana Bell,Douglas Elliman,Lana Bell,Douglas Elliman,Chateau Beach 17475 Collins Ave 1901 Sunny Isl...,Chateau Beach,17475 Collins Ave 1901 Sunny Isles Beach,"(25.9378917, -80.12093329999999)",25.937892,-80.120933,3300000.0,4,blue
5,Murano Grande\r400 Alton Rd 1201\rMiami Beach,5/3/2023,264,"$3,200,000.00",2003,1597.60,Billy Hernandez,Douglas Elliman,Nancy Batchelor PA,"Compass Florida, LLC",Murano Grande 400 Alton Rd 1201 Miami Beach,Murano Grande,400 Alton Rd 1201 Miami Beach,"(25.7731379, -80.14060429999999)",25.773138,-80.140604,3200000.0,5,blue
6,Lake Villa One Condo\r721 Crandon Blvd PH8\rKe...,5/1/2023,122,"$2,245,000.00",2683,836.75,Coralia Braverman,Florida Capital Realty,Michelle Shurtleff,"Compass Florida, LLC",Lake Villa One Condo 721 Crandon Blvd PH8 Key ...,Lake Villa One Condo,721 Crandon Blvd PH8 Key Biscayne,"(25.6879859, -80.16285909999999)",25.687986,-80.162859,2245000.0,6,blue
7,Carbonell Condo\r901 Brickell Key Blvd 3708\rM...,5/3/2023,24,"$2,102,000.00",2860,734.97,April Feeser,Realhome Service & Solutions I,Estela Serrano,"Serrano & Associates Real Estate, LLC",Carbonell Condo 901 Brickell Key Blvd 3708 Miami,Carbonell Condo,901 Brickell Key Blvd 3708 Miami,"(25.7691361, -80.1869402)",25.769136,-80.186940,2102000.0,7,blue
8,TDR Tower II Condo\r15901 Collins Ave 4104\rSu...,5/3/2023,182,"$2,100,000.00",2327,902.45,Olena Sartan,Miami Residence Inc.,Marco Cunha,Prestige Luxury Real Estate,TDR Tower II Condo 15901 Collins Ave 4104 Sunn...,TDR Tower II Condo,15901 Collins Ave 4104 Sunny Isles Beach,"(25.9240235, -80.1218071)",25.924024,-80.121807,2100000.0,8,blue
9,The Pinnacle Condo\r17555 Collins Ave PH-7\rSu...,5/1/2023,32,"$2,000,000.00",1840,1086.96,Tomer Kort,"Brokerage S. Florida Real Estate, LLC.",Hector Stracuzzi,Beachfront Realty Inc,The Pinnacle Condo 17555 Collins Ave PH-7 Sunn...,The Pinnacle Condo,17555 Collins Ave PH-7 Sunny Isles Beach,"(25.9392319, -80.120541)",25.939232,-80.120541,2000000.0,9,blue
10,02977 Bridgeport Condo\r2977 Bridgeport Ave 29...,5/1/2023,56,"$1,965,000.00",2802,701.28,Monique Pennen-Wills PA,BHHS EWM Realty,Axelle Gault,"Global Axellence, Inc",02977 Bridgeport Condo 2977 Bridgeport Ave 297...,Bridgeport,2977 Brigeport Ave,"(25.7360738, -80.2431314)",25.736074,-80.243131,1965000.0,10,blue


In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [18]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Grovenor House , 2627 S Bayshore Dr 2403 Coconut Grove | Price $4,250,000 | $1,610 psf | Listing agent: Anthony Lauria with Compass Florida, LLC | Buyer's agent: Geane Brito with Compass Florida, LLC | Days on market: 16
Least Expensive
Bridgeport, 2977 Brigeport Ave | Price $1,965,000 | $701 psf | Listing agent: Monique Pennen-Wills PA with BHHS EWM Realty | Buyer's agent: Axelle Gault with Global Axellence, Inc | Days on market: 56
Highest Price Per Square Foot
Palau Sunset Harbour , 1201 20th St 205 Miami Beach | Price $3,650,000 | $2,005 psf | Listing agent: John Sandberg with Compass Florida, LLC | Buyer's agent: Salesperson Non Member with Non-MLS Member | Days on market: 1
Lowest Price Per Square Foot
Bridgeport, 2977 Brigeport Ave | Price $1,965,000 | $701 psf | Listing agent: Monique Pennen-Wills PA with BHHS EWM Realty | Buyer's agent: Axelle Gault with Global Axellence, Inc | Days on market: 56
Most Days on Market
Murano Grande , 400 Alton Rd 1201 Miami Beach 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_05062023
